In [1]:
import numpy as np
import pandas as pd

In [2]:
neuron_df = pd.read_csv('Data/neurons.csv')
connection_df = pd.read_csv('Data/connections.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/neurons.csv'

In [19]:
non_conns = ['NO_CONS']
center_regions = ['NO', 'PB', 'EB', 'FB',   # Central Complex: Involved in multiple complex behaviors
                  'SAD', 'PRW',             # Periesophageal Neuropils Center: Sensory (including gustatory) processing, motor control, and potentially circadian rhythm.
                  'GNG',                    # controlling the mouthparts, feeding behavior and taste info
                  'OCG',                    # Ocelli: contribute to light detection and circadian rhythm
                  'UNASGD']                 # Unassigned: no known function
left_right_regions = ['AME', 'LA', 'LO', 'LOP', 'ME',       # Optic lobe: Visual sensory region
                      'BU', 'LAL', 'GA',                    # Lateral Complex: Integration of sensory information
                      'LH',                                 # Lateral Horn: Olfactory sensory region
                      'CAN', 'AMMC', 'FLA',                 # Periesophageal Neuropils: Gustatory sensory region, motor control
                      'ICL', 'IB', 'ATL', 'CRE', 'SCL',     # Inferior Neuropils: Sensory integration, motor control
                      'VES', 'GOR', 'SPS', 'IPS', 'EPA',    # Ventromedial Neuropils: Sensory integration, learning and memory
                      'MB_PED', 'MB_VL', 'MB_ML', 'MB_CA',  # Mushroom Body: Olfactory learning and memory
                      'AL',                                 # Antennal Lobe: Olfactory sensory region
                      'SLP', 'SIP', 'SMP',                  # Superior Neuropils: Integration of multi-modality sensory information
                      'AVLP', 'PVLP', 'WED', 'PLP', 'AOTU'] # Ventrolateral Neuropils: Sensory processing and cognitive functions

In [32]:
# Drop the columns that are not needed
neuron_df = neuron_df.drop(columns=["nt_type_score", "da_avg", "ser_avg", "gaba_avg", 'glut_avg', 'ach_avg', 'oct_avg'])

# Sort the data by the group
neuron_df = neuron_df.sort_values(by=['group'])

# Batch edit the value of group column based on regex: delete the string after the first dot
neuron_df['group'] = neuron_df['group'].str.replace(r'\..*', '')

# Rename the column name of connection_df pre_root_id to neuron_id
neuron_df = neuron_df.rename(columns={'root_id': 'neuron_id'})
connection_df = connection_df.rename(columns={'pre_root_id': 'neuron_id'})

combine_df = pd.merge(connection_df, neuron_df, on='neuron_id', how='left')
combine_df = combine_df.rename(columns={'group': 'pre_group'})
combine_df = combine_df.rename(columns={'nt_type_x': 'syn_type'})
combine_df = combine_df.rename(columns={'nt_type_y': 'nt_type'})

/var/folders/gt/29lqqd_s6x94hjy8ys1rt98w0000gn/T/ipykernel_26830/603410581.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  neuron_df['group'] = neuron_df['group'].str.replace(r'\..*', '')


In [33]:
neuron_conn_df = pd.merge(neuron_df, connection_df, on='neuron_id', how='right')
neuron_conn_df = neuron_conn_df.rename(columns={'group': 'pre_group'})
neuron_conn_df = neuron_conn_df.rename(columns={'nt_type_y': 'syn_type'})
neuron_conn_df = neuron_conn_df.rename(columns={'nt_type_x': 'nt_type'})

In [35]:
neuron_df

,neuron_id,group,nt_type
65780,720575940625030920,AL,SER
103884,720575940633175196,AL,ACH
3894,720575940606949042,AL,ACH
27153,720575940616815005,AL,ACH
54429,720575940622745834,AL,ACH
...,...,...,...
123110,720575940644252320,WED,ACH
121883,720575940642627876,WED,ACH
44333,720575940620862746,WED,ACH
51063,720575940622135757,WED,ACH


In [34]:
region_conn_info = neuron_conn_df.loc[neuron_conn_df['pre_group'] == 'NO_CONS']
region_conn_info = region_conn_info.sort_values(by=['neuron_id'])
region_neural_info = region_conn_info.drop_duplicates(subset=['neuron_id'])
region_neural_info
# 720575940613437206
# neuron_count = region_conn_info['neuron_id'].value_counts()
# neuron_count = neuron_count.sort_index()
# neuron_count_array = neuron_count.to_numpy()
# l = []
# for i in range(len(neuron_count)):
#     if neuron_count_array[i] == 1 and pd.isnull(region_conn_info.loc[region_conn_info['neuron_id']==neuron_count.index[i]].syn_count).item():
#         neuron_count_array[i] = 0

# neuron_count_array

,neuron_id,pre_group,nt_type,post_root_id,neuropil,syn_count,syn_type


In [123]:
Neurons_dict = {}
Conn_dict = {}

for region in center_regions + left_right_regions + non_conns:
    region_info = neuron_df.loc[neuron_df['group'] == region]
    region_info = region_info.sort_values(by=['neuron_id'])
    # delete all the duplicate neuron_id row
    region_neural_info = region_info.drop_duplicates(subset=['neuron_id'])
    # Neuron Information
    Neurons_dict[region] = {}
    Neurons_dict[region]['neuron_number'] = len(region_neural_info['neuron_id'])
    Neurons_dict[region]['neuron_ids'] = region_neural_info['neuron_id'].to_numpy()
    Neurons_dict[region]['nt_type'] = region_neural_info['nt_type'].to_numpy()

    # Connection Information
    region_conn_info = neuron_conn_df.loc[neuron_conn_df['pre_group'] == region]
    region_conn_info = region_conn_info.sort_values(by=['neuron_id'])
    Conn_dict[region] = {}
    Conn_dict[region]['conn_number'] = len(region_conn_info['neuron_id'])
    Conn_dict[region]['pre_ids'] = region_conn_info['neuron_id'].to_numpy()
    Conn_dict[region]['post_ids'] = region_conn_info['post_root_id'].to_numpy()
    neuron_count = region_conn_info['neuron_id'].value_counts()
    neuron_count = neuron_count.sort_index()
    indptr = neuron_count.to_numpy()
    Conn_dict[region]['indptr'] = indptr
    Conn_dict[region]['syn_count'] = region_conn_info['syn_count'].to_numpy()
    region_conn_info['nt_type'] = region_conn_info['nt_type'].fillna(region_conn_info['syn_type'])
    Conn_dict[region]['syn_type'] = region_conn_info['nt_type'].to_numpy()

import pickle
with open('Data/neuron.pkl', 'wb') as f:
	pickle.dump(Neurons_dict, f)

with open('Data/conn.pkl', 'wb') as f:
    pickle.dump(Conn_dict, f)

In [105]:
sum = 0
for region in center_regions + left_right_regions:
    sum += Neurons_dict[region]['neuron_number']

print(sum)
print(len(neuron_df['neuron_id']))

121327
127978


In [120]:
region_info = neuron_df.loc[neuron_df['group'] == 'BU']
region_info = region_info.sort_values(by=['neuron_id'])
region_neural_info = region_info.drop_duplicates(subset=['neuron_id'])
region_info

,neuron_id,group,nt_type
23586,720575940615746677,BU,GABA
96451,720575940631242002,BU,NaN


In [115]:
region_neural_info = neuron_df.loc[neuron_df['group'] == 'NO_CONS']
region_neural_info

,neuron_id,group,nt_type
1802,720575940605364448,NO_CONS,NaN
11663,720575940611759569,NO_CONS,NaN
10049,720575940611090478,NO_CONS,NaN
127464,720575940657606529,NO_CONS,ACH
125357,720575940647019043,NO_CONS,NaN
...,...,...,...
110443,720575940635837786,NO_CONS,SER
20971,720575940614884754,NO_CONS,NaN
20927,720575940614865002,NO_CONS,NaN
21197,720575940614957115,NO_CONS,NaN


In [128]:
region_conn_info = neuron_conn_df.loc[neuron_conn_df['pre_group'] == 'OCG']
region_conn_info = region_conn_info.sort_values(by=['neuron_id'])
# region_neural_info = region_conn_info.drop_duplicates(subset=['neuron_id'])
region_conn_info['nt_type'].unique()

array([nan, 'SER', 'ACH', 'GLUT', 'GABA'], dtype=object)

In [90]:
print(len(region_info['neuron_id']))
neuron_count = region_info['neuron_id'].value_counts()
neuron_count = neuron_count.sort_index()
neuron_count.to_numpy()

369342


array([13, 10, 14, ...,  1, 41, 26])

In [91]:
region_info['post_root_id']

0          720575940608552405
12         720575940640902389
11         720575940635713375
10         720575940634848875
9          720575940630768215
                  ...        
3794553    720575940607969289
3794552    720575940606156832
3794576    720575940631129362
3794563    720575940621647948
3794577    720575940632227155
Name: post_root_id, Length: 369342, dtype: int64

In [93]:
region_info['syn_count']

0           5
12         16
11          5
10         17
9           5
           ..
3794553    12
3794552    43
3794576    12
3794563     2
3794577     8
Name: syn_count, Length: 369342, dtype: int64

In [94]:
region_info

,neuron_id,pre_group,nt_type,post_root_id,neuropil,syn_count,syn_type
0,720575940596125868,LO,ACH,720575940608552405,LOP_R,5,ACH
12,720575940596125868,LO,ACH,720575940640902389,LOP_R,16,ACH
11,720575940596125868,LO,ACH,720575940635713375,LOP_R,5,ACH
10,720575940596125868,LO,ACH,720575940634848875,LOP_R,17,ACH
9,720575940596125868,LO,ACH,720575940630768215,LOP_R,5,ACH
...,...,...,...,...,...,...,...
3794553,720575940660863105,LO,GLUT,720575940607969289,LO_L,12,GLUT
3794552,720575940660863105,LO,GLUT,720575940606156832,LO_L,43,GLUT
3794576,720575940660863105,LO,GLUT,720575940631129362,AOTU_L,12,GLUT
3794563,720575940660863105,LO,GLUT,720575940621647948,SLP_L,2,GLUT
